In [1]:
import pandas as pd
import pyarrow.parquet as pp

import gc

In [3]:
db = pd.read_parquet("/home/sergey/projects/Credit_scoring/raw/train_data/train_data_1.pq", engine="pyarrow")
db[:5]

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,...,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag
0,250000,1,8,7,7,2,4,9,5,2,...,3,3,3,4,1,3,4,1,0,0
1,250000,2,14,8,12,10,6,3,12,6,...,3,3,3,4,1,3,4,1,0,0
2,250000,3,10,4,8,11,6,13,5,2,...,3,3,3,4,1,3,4,1,0,0
3,250000,4,13,12,8,6,13,5,12,2,...,3,3,3,4,1,3,4,1,0,0
4,250000,5,6,12,4,14,1,7,4,2,...,3,3,3,4,1,3,1,1,1,0


In [4]:
gc.collect()

1177